In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
import gpiozero
import time

In [2]:
# Defining GPIO inputs and outputs
pwm_l = gpiozero.PWMOutputDevice(pin=14,active_high=True,initial_value=0,frequency=50000)
dir_l = gpiozero.OutputDevice(pin=23)

pwm_r = gpiozero.PWMOutputDevice(pin=15,active_high=True,initial_value=0,frequency=50000)
dir_r = gpiozero.OutputDevice(pin=24)

dir_l = 1
dir_r = 1

encoder_l = gpiozero.RotaryEncoder(a=2, b=3,max_steps=100000) 
encoder_r = gpiozero.RotaryEncoder(a=17, b=27,max_steps=100000) 

# add sensors

In [3]:
# Defining robot model with real feedback
class DiffDriveRobot:
    
    def __init__(self, inertia=5, dt=0.1, drag=1, wheel_radius=0.028, wheel_sep=0.101):
        
        self.x = 0.0 # y-position
        self.y = 0.0 # y-position 
        self.th = 0.0 # orientation
        
        self.wl = 0.0 #rotational velocity left wheel
        self.wr = 0.0 #rotational velocity right wheel
        
        self.I = inertia
        self.d = drag
        self.dt = dt
        
        self.r = wheel_radius
        self.l = wheel_sep
    
    # Calculate motor speed from measured shaft encoder feedback
    def motor_speed(self,encoder):
                
        N = 32 # 32 line encoder
        GEAR_RATIO = 114.7 
        steps = encoder.steps # encoder feedback
        rps = (steps/self.dt)/N
        rpm = rps*60
        wheel_rpm = rpm/GEAR_RATIO
        w_min = (2*np.pi*self.r*wheel_rpm)/1000
        w = w_min/60
        
#         torque = self.I*duty_cycle
#         w = w + self.dt*(torque - self.d*w)

        
        return w
    
    # Velocity motion model
    # Base linear and angular velocity from actual left and right angular velocity
    def base_velocity(self,wl,wr):
        
        v = (wl*self.r + wr*self.r)/2.0
        
        w = (wl - wr)/self.l
        
        return v, w
    
    # Kinematic motion model
    def pose_update(self,duty_cycle_l,duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r):
        
        self.wl = self.motor_speed(encoder_l)
        self.wr = self.motor_speed(encoder_r)
        
        v, w = self.base_velocity(self.wl,self.wr)
        
        pwm_l.value = duty_cycle_l
        pwm_r.value = duty_cycle_r
        
        self.x = self.x + self.dt*v*np.cos(self.th)
        self.y = self.y + self.dt*v*np.sin(self.th)
        self.th = self.th + w*self.dt
        
        
        return self.x, self.y, self.th
        

In [4]:
# Defining robot velocity controller
class RobotController:
    
    def __init__(self, Kp=1, Ki=0.25, wheel_radius=0.028, wheel_sep=0.101):
        
        self.Kp = Kp
        self.Ki = Ki
        self.r = wheel_radius
        self.l = wheel_sep
        self.e_sum_l = 0 # error term of left wheel
        self.e_sum_r = 0 # error term of right wheel
        
    # Implement duty cycle PI control given target and actual angular velocity
    def p_control(self,w_desired,w_measured,e_sum):
        
        duty_cycle = min(max(-1,self.Kp*(w_desired-w_measured) + self.Ki*e_sum),1)
        
        e_sum = e_sum + (w_desired-w_measured)
        
        return duty_cycle, e_sum
        
        
    # Required duty cycle from target linear and angular velocity 
    def drive(self,v_desired,w_desired,wl,wr):
        
        wl_desired = v_desired/self.r + self.l*w_desired/2 
        wr_desired = v_desired/self.r - self.l*w_desired/2
        
        duty_cycle_l,self.e_sum_l = self.p_control(wl_desired,wl,self.e_sum_l)
        duty_cycle_r,self.e_sum_r = self.p_control(wr_desired,wr,self.e_sum_r)
        
        
        return duty_cycle_l, duty_cycle_r
        

In [5]:
# Initalising models
robot = DiffDriveRobot(inertia=5, dt=0.1, drag=1, wheel_radius=0.028, wheel_sep=0.101)
controller = RobotController(Kp=1, Ki=0.25, wheel_radius=0.028, wheel_sep=0.101)

In [6]:
# Example controller

poses = []
velocities = []
duty_cycle_commands = []

for i in range(300):
    
    if i < 100: # drive in circular path (turn left) for 10 s
        duty_cycle_l,duty_cycle_r = controller.drive(0.1,1,robot.wl,robot.wr)
    elif i < 200: # drive in circular path (turn right) for 10 s
        duty_cycle_l,duty_cycle_r = controller.drive(0.1,-1,robot.wl,robot.wr)
    else: # stop
        duty_cycle_l,duty_cycle_r = (0,0)
    
    # Drive robot motion - send duty cycle command to robot
    x,y,th = robot.pose_update(duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r)

    # Log data
    poses.append([x,y,th])
    duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
    velocities.append([robot.wl,robot.wr])

In [7]:
# Defining robot path planner (without obstacle avoidance)


In [8]:
# Defining robot path planner (with obstacle avoidance)

In [9]:
# Define robot coordinates

In [10]:
# Run robot using predefined coordinates

In [11]:
# Run robot indefinitely for final competition